In [1]:
# Import packages
import pyspark
import dxpy
import dxdata
from pyspark.sql import SQLContext

In [2]:
# Spark initialization (Done only once; do not rerun this cell unless you select Kernel -> Restart kernel).
# Need to adjust this buffer otherwise will get an error in toPandas() call
conf = pyspark.SparkConf().set("spark.kryoserializer.buffer.max", "128m")
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.sql.SparkSession(sc)
sqlContext = SQLContext(sc)

/cluster/spark/python/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
# Automatically discover dispensed database name and dataset id
dispensed_database = dxpy.find_one_data_object(
    classname='database', 
    name='app*', 
    folder='/', 
    name_mode='glob', 
    describe=True)
dispensed_database_name = dispensed_database['describe']['name']

dispensed_dataset = dxpy.find_one_data_object(
    typename='Dataset', 
    name='app*.dataset', 
    folder='/', 
    name_mode='glob')
dispensed_dataset_id = dispensed_dataset['id']

In [4]:
dataset = dxdata.load_dataset(id=dispensed_dataset_id)

In [13]:
field_name_dict = {
    'sample_names': 'eid',
    'hes_info': 'p41259',
}

In [8]:
field_names = list(field_name_dict.values())

In [9]:
participant = dataset['participant']

In [10]:
df = participant.retrieve_fields(names=field_names, engine=dxdata.connect(), coding_values="raw")

In [12]:
def upload_file_to_project(filename, proj_dir):
    dxpy.upload_local_file(filename, folder=proj_dir, parents=True)
    print(f"*********{filename} uploaded!!*********")
    return

In [14]:
pandas_df = df.toPandas()

In [15]:
pandas_df = pandas_df.rename(columns={v:k for k,v in field_name_dict.items()})

In [17]:
proj_dir = "/fields/data/hes_info/"
filename = "hes_info.csv.gz"
pandas_df.to_csv(filename, index=False)
upload_file_to_project(filename, proj_dir)

*********hes_info.csv.gz uploaded!!*********


In [11]:
# See the first five entries as a Pandas DataFrame:
df.limit(5).toPandas()

,eid,p41259
0,2538818,18
1,5093299,15
2,4244964,73
3,3351884,1
4,5632043,2
